## Setup

In [1]:
import init_load_board
exec(open('init_load_board.py').read())

import init_simple_mdp
exec(open('init_simple_mdp.py').read())

import numpy as np

import function_board as fb 
import function_tool as ft
import function_get_aiming_grid 
exec(open('function_get_aiming_grid.py').read())

import evaluate_score_probability_with_error as esp
import function_solve_dp

#%%
data_parameter_dir = fb.data_parameter_dir
result_dir = './result'       



a_throw_list = []
a_token_list = []

for a in actions:
    a_throw_list.append(a)

for a in token_actions:
    a_token_list.append(a)

a_list = a_throw_list + a_token_list
throw_num = len(a_throw_list)

name_pa = 'player{}'.format(10)


import os
import sys
import time

import function_board as fb
import function_tool as ft
import function_get_aiming_grid
import function_evaluate_policy as fep

import numpy as np
np.set_printoptions(precision=4)
np.set_printoptions(linewidth=300)
np.set_printoptions(threshold=300)

import torch
torch.set_printoptions(precision=4)
torch.set_printoptions(linewidth=300)
torch.set_printoptions(threshold=300)

[aiming_grid, prob_grid_normalscore, prob_grid_singlescore, prob_grid_doublescore, prob_grid_triplescore, prob_grid_bullscore] = function_get_aiming_grid.load_aiming_grid(name_pa, data_parameter_dir=data_parameter_dir, grid_version='custom_no_tokens')

[aiming_grid, prob_grid_normalscore_nt, prob_grid_singlescore_nt, prob_grid_doublescore_nt, prob_grid_triplescore_nt, prob_grid_bullscore_nt] = function_get_aiming_grid.load_aiming_grid(name_pa, data_parameter_dir=data_parameter_dir, grid_version='custom_no_tokens')
[aiming_grid, prob_grid_normalscore_t, prob_grid_singlescore_t, prob_grid_doublescore_t, prob_grid_triplescore_t, prob_grid_bullscore_t] = function_get_aiming_grid.load_aiming_grid(name_pa, data_parameter_dir=data_parameter_dir, grid_version='custom_tokens')

import function_solve_dp_tokens as fsdt

load_pickle from ./data_parameter/player_gaussin_fit/grid_custom_no_tokens/player10_gaussin_prob_grid_custom_no_tokens.pkl
load_pickle from ./data_parameter/player_gaussin_fit/grid_custom_no_tokens/player10_gaussin_prob_grid_custom_no_tokens.pkl
load_pickle from ./data_parameter/player_gaussin_fit/grid_custom_tokens/player10_gaussin_prob_grid_custom_tokens.pkl


In [47]:
# #prob_notbust
# prob_notbust = np.zeros(len(prob_grid_normalscore_nt[:,0]))
# prob_notbust[:throw_num] += prob_grid_normalscore_nt[:throw_num,0:4+1].sum(axis=1)
# prob_notbust[:throw_num] += prob_grid_normalscore_t[throw_num:,0:4+1].sum(axis=1)

# prob_notbust

In [2]:
def solve_turn_transit_probability_nt(score_state, state_action, prob_normalscore, prob_doublescore, prob_bullscore):
    """
    Solve the state transition probability after a turn playing with a specified aiming policy
    
    Args: 
        score_state: score at the beginning of the turn, e.g., 2,3,...,501 
        state_action: a dict of aiming locations (actions in the policy) for each state (s,i,u) in this turn
        prob_normalscore, prob_doublescore, prob_bullscore: the skill model 
    
    Returns: A dict
        result_dict['finish']: probability of finishing the game (reach zero by making a double)
        result_dict['bust']: probability of busting the game (transit to next turn of (s=score_state,i=3,u=0))
        result_dict['score']: probability of achieving a cumulative score_gained in this turn (transit to next turn of (s=score_state-score_gained,i=3,u=0))
    """    
    
    ##
    result_dict = {}
    prob_finish = 0 ## probability of finishing the game
    prob_bust = 0   ## probability of busting the game
    ## initialize for (s, rt=3, score_gained=0)
    next_throw_state_len = 1
    prob_transit_next_throw_state = np.ones(next_throw_state_len)
    
    for rt in [3,2,1]:
        prob_this_throw_state = prob_transit_next_throw_state
        this_throw_state_len = next_throw_state_len
        next_throw_state_len = min(score_state-2, fb.maxhitscore*(4-rt)) + 1
        prob_transit_next_throw_state = np.zeros(next_throw_state_len)  ## probability vector of total score_gained after this throw
        
        for score_gained in range(this_throw_state_len):
            ## skip infeasible state
            if not fb.state_feasible_array[rt, score_gained]:
                continue   

            ## aimming location of the policy at this state
            aiming_location_index = state_action[rt][0][score_gained]
            prob_this_state = prob_this_throw_state[score_gained]
            
            #largest possible normal socre to make in the next throw without busting
            score_remain = score_state - score_gained
            score_max = min(score_remain-2, 60)
            score_max_plus1 = score_max + 1
        
            ## transit to next throw or turn with normal scores
            prob_transit_next_throw_state[score_gained:score_gained+score_max_plus1] += prob_normalscore[aiming_location_index, 0:score_max_plus1]*prob_this_state
            ## game can not bust or end when score_max = 60, i.e.,  prob_notbust = 1
            if (score_max < 60):
                prob_notbust_this_state = prob_normalscore[aiming_location_index, 0:score_max+1].sum()
                ## transit to the end of game
                if (score_remain == fb.score_DB):
                    prob_finish += prob_bullscore[aiming_location_index, 1]*prob_this_state
                    prob_notbust_this_state += prob_bullscore[aiming_location_index, 1]
                elif (score_remain <= 40 and score_remain%2==0):
                    doublescore_index = (score_remain//2) - 1
                    prob_finish += prob_doublescore[aiming_location_index, doublescore_index]*prob_this_state
                    prob_notbust_this_state += prob_doublescore[aiming_location_index, doublescore_index]
                else:
                    pass

                ## transit to bust
                prob_bust += (max(1 - prob_notbust_this_state,0))*prob_this_state
            
    result_dict['finish'] = prob_finish
    result_dict['bust'] = prob_bust
    result_dict['score'] = prob_transit_next_throw_state

    return result_dict


In [3]:
def solve_turn_transit_probability_fast_nt(score_state, state_action, prob_normalscore_nt, prob_normalscore_t, prob_doublescore_nt,prob_doublescore_t, prob_bullscore_nt, prob_bullscore_t, prob_bust_dic_nt, prob_bust_dic_t):
    """
    A fast way of implementing solve_turn_transit_probability by using pre-stored prob_bust_dic
    """     
    
    result_dict = {}
    prob_finish = 0 ## probability of finishing the game
    prob_bust_total = 0   ## probability of busting the game
    ## initialize for (s, rt=3, score_gained=0)
    next_throw_state_len = 1
    prob_transit_next_throw_state = np.ones(next_throw_state_len)
    
    for rt in [3,2,1]:
        prob_this_throw_state = prob_transit_next_throw_state
        this_throw_state_len = next_throw_state_len
        next_throw_state_len = min(score_state-2, fb.maxhitscore*(4-rt)) + 1
        prob_transit_next_throw_state = np.zeros(next_throw_state_len)  ## probability vector of total score_gained after this throw
        
        prob_normalscore_transit = prob_normalscore_nt[state_action[rt][0][0:this_throw_state_len]]*prob_this_throw_state.reshape((this_throw_state_len,1))
        
        for score_gained in range(this_throw_state_len):  # loop through score already gained
            ## skip infeasible state
            if not fb.state_feasible_array[rt, score_gained]:
                continue   

            ## aimming location of the policy at this state
            aiming_location_index = state_action[rt][0][score_gained]
            prob_this_state = prob_this_throw_state[score_gained]
            
            #largest possible normal socre to make in the next throw without busting
            score_remain = score_state - score_gained
            score_max = min(score_remain-2, 60)
            score_max_plus1 = score_max + 1
        
            ## transit to next throw or turn with normal scores            
            prob_transit_next_throw_state[score_gained:score_gained+score_max_plus1] += prob_normalscore_transit[score_gained, 0:score_max_plus1]
            ## game can not bust or end when score_max = 60, i.e.,  prob_notbust = 1
            if (score_max < 60):
                ## transit to the end of game
                if (score_remain == fb.score_DB):
                    prob_finish += prob_bullscore_nt[aiming_location_index, 1]*prob_this_state
                elif (score_remain <= 40 and score_remain%2==0):
                    doublescore_index = (score_remain//2) - 1
                    prob_finish += prob_doublescore_nt[aiming_location_index, doublescore_index]*prob_this_state
                else:
                    pass

                #transit to bust
                prob_bust_total += prob_bust_dic_nt[score_max][aiming_location_index]*prob_this_state
            
    result_dict['finish'] = prob_finish
    result_dict['bust'] = prob_bust_total
    result_dict['score'] = prob_transit_next_throw_state

    return result_dict 

In [4]:
def solve_policy_transit_probability_nt(policy_action_index_dic, prob_grid_normalscore, prob_grid_doublescore, prob_grid_bullscore):
    """
    For each turn, solve the state transition probability for a specified aiming policy
    
    Args: 
        policy_action_index_dic: a dict of aiming locations (actions in the policy) for each state (s,i,u) of each turn s=2,...,501
        prob_normalscore, prob_doublescore, prob_bullscore: the skill model 
    
    Returns: A dict
    """  
    
    prob_policy_transit_dict = {}
    t1 = time.time()
    for score_state in range(2,502):
        prob_policy_transit_dict[score_state] = solve_turn_transit_probability_nt(score_state, policy_action_index_dic[score_state], prob_grid_normalscore, prob_grid_doublescore, prob_grid_bullscore)

    t2 = time.time()
    print('solve prob_policy_transit in {} seconds'.format(t2-t1))
    
    return prob_policy_transit_dict


# Token Code

In [5]:
tokens = 9
max_token_index = tokens + 1

In [11]:
#pzerolist = []
turn_transit_list = {}
## aiming_grid
num_aiming_location = aiming_grid.shape[0]

prob_normalscore_t = prob_grid_normalscore_t
prob_doublescore_dic_t = {}

prob_normalscore_nt = prob_grid_normalscore_nt
prob_doublescore_dic_nt = {}

# Initialize the probabilities of hitting a double (both for token and no token)
for doublescore_index in range(20):
    doublescore = 2*(doublescore_index+1)
    # Tokens 
    doublescore_dict_array_t = np.zeros(len(prob_grid_normalscore_nt[:,0]))
    doublescore_dict_array_t[throw_num:] = prob_grid_doublescore_t[throw_num:,doublescore_index]
    prob_doublescore_dic_t[doublescore] = doublescore_dict_array_t

    # No Tokens 
    doublescore_dict_array_nt = np.zeros(len(prob_grid_normalscore_nt[:,0]))
    doublescore_dict_array_nt[:throw_num] = prob_grid_doublescore_t[:throw_num,doublescore_index]
    prob_doublescore_dic_nt[doublescore] = doublescore_dict_array_nt

prob_DB_nt = np.array(prob_grid_bullscore_nt[:,1])
prob_DB_t = np.array(prob_grid_bullscore_t[:,1])

## the probability of not bust for each action given score_max=i (score_remain=i+2)
prob_bust_dic_t = {}
prob_notbust_dic_t = {}

prob_bust_dic_nt = {}
prob_notbust_dic_nt = {}

for score_max in range(60):  

    ## transit to next throw or turn
    prob_notbust_t = np.zeros(len(prob_grid_normalscore_nt[:,0]))
    # prob_notbust_t[:throw_num] += prob_grid_normalscore_nt[:throw_num,0:score_max+1].sum(axis=1)
    prob_notbust_t[throw_num:] += prob_grid_normalscore_t[throw_num:,0:score_max+1].sum(axis=1) 

    #prob_notbust_nt = np.zeros(len(prob_grid_normalscore_nt[:,0]))
    # prob_notbust_t[:throw_num] += prob_grid_normalscore_nt[:throw_num,0:score_max+1].sum(axis=1)
    #prob_notbust_nt[:throw_num] += prob_grid_normalscore_nt[:throw_num,0:score_max+1].sum(axis=1) 
    prob_notbust_nt = prob_grid_normalscore_nt[:,0:score_max+1].sum(axis=1) 

    ## transit to the end of game
    score_remain = score_max + 2

    if (score_remain == fb.score_DB):
        prob_notbust_nt += prob_DB_nt
        # prob_notbust_t += prob_DB_nt
        prob_notbust_t += prob_DB_t
    elif (score_remain <= 40 and score_remain%2==0):
        prob_notbust_nt += prob_doublescore_dic_nt[score_remain]
        # prob_notbust_t += prob_doublescore_dic_nt[score_remain]
        prob_notbust_t += prob_doublescore_dic_t[score_remain]
         
    ## Normalize to desired probability range 
    prob_notbust_t = np.minimum(np.maximum(prob_notbust_t, 0),1)
    prob_notbust_nt = np.minimum(np.maximum(prob_notbust_nt, 0),1)
    
    # Append to final dictionaries
    prob_notbust_dic_t[score_max] = prob_notbust_t
    prob_bust_dic_t[score_max] = 1 - prob_notbust_dic_t[score_max]

    prob_notbust_dic_nt[score_max] = prob_notbust_nt
    prob_bust_dic_nt[score_max] = 1 - prob_notbust_dic_nt[score_max]

######################
prob_bust_dic_nt = prob_bust_dic
prob_DB_nt = prob_DB
prob_doublescore_dic_nt = prob_doublescore_dic
prob_normalscore_nt = prob_normalscore
prob_notbust_dic_nt = prob_notbust_dic
######################

# Add to tensors
prob_normalscore_tensor_nt = torch.from_numpy(prob_normalscore_nt)
prob_normalscore_tensor_t = torch.from_numpy(prob_normalscore_t)

iteration_round_limit = 20
iteration_relerror_limit = 10**-9

#### state space example of (SB=25 DB=50) ####
## rt: the number of remaining throws in a turn
## state_infeasible_rt2 = [23, 29, 31, 35, 37, 41, 43, 44, 46, 47, 49, 52, 53, 55, 56, 58, 59]
## state_infeasible_rt1 = [103, 106, 109, 112, 113, 115, 116, 118, 119]    

optimal_value_rt3 = np.zeros((max_token_index,502))
optimal_value_dic = {}
optimal_action_index_dic= {}

num_iteration_record = np.zeros((max_token_index,502), dtype=np.int32)

state_len_vector = np.zeros(4, dtype=np.int32)
state_value  = [None]  ## optimal value (expected # of turns to finish the game) for each state in the current playing turn
state_action = [None]  ## aimming locations for for each state in the current playing turn
action_diff  = [None]
value_relerror = np.zeros((max_token_index,4))

for rt in [1,2,3]:

    ## for rt=3: possible score_gained = 0
    ## for rt=2: possible score_gained = 0,1,...,60
    ## for rt=1: possible score_gained = 0,1,...,120

    # Number of possible scores for given remaining throws (rt) in the turn
    this_throw_state_len = fb.maxhitscore*(3-rt) + 1

    # Create stacked state value, action and action diff objects 
    # Rows will be token values, columns will be remaining scores 
    state_value.append(np.ones((max_token_index,this_throw_state_len))*fb.largenumber)
    state_action.append(np.ones((max_token_index,this_throw_state_len), np.int32)*fb.infeasible_marker)
    action_diff.append(np.ones((max_token_index,this_throw_state_len)))

# Add these to the state value update objects 
state_value_update = ft.copy_numberarray_container(state_value)
state_action_update = ft.copy_numberarray_container(state_action)

## use no_turn policy as the initial policy 
[noturn_optimal_value, noturn_optimal_action_index] = fsdt.solve_dp_noturn_tokens(aiming_grid, prob_grid_normalscore_nt, prob_grid_normalscore_t, tokens = 0, prob_grid_doublescore=prob_grid_doublescore_nt, prob_grid_bullscore=prob_grid_bullscore_nt,prob_grid_doublescore_t=prob_grid_doublescore_t, prob_grid_bullscore_t=prob_grid_bullscore_t)

t1 = time.time()

for t in range(0,max_token_index):

    if t==0:

        for score_state in range(2, 502):

            turn_transit_list[score_state] = []

            # if score_state == 3:
            #     break 
            #print('#### solve_dp_turn score_state={} ####'.format(score_state))    
            
            ## initialization 
            for rt in [1,2,3]:
                ## for rt=3: score_gained = 0
                ## for rt=2: score_gained = 0,1,...,min(s-2,60)
                ## for rt=1: score_gained = 0,1,...,min(s-2,120)
                this_throw_state_len = min(score_state-2, fb.maxhitscore*(3-rt)) + 1
                state_len_vector[rt] = this_throw_state_len
                        
                ## initialize the starting policy: 
                ## use no_turn action in (s, i, u=0)
                ## use turn action (s-1, i, u-1) in (s, i, u!=0) if (s-1, i, u-1) is feasible state
                state_action[rt][0] = noturn_optimal_action_index[0,score_state]            
                for score_gained in range(1,this_throw_state_len):                
                    if fb.state_feasible_array[rt, score_gained]:  ## if True
                        if fb.state_feasible_array[rt, score_gained-1]:
                            state_action[rt][0][score_gained] = optimal_action_index_dic[score_state-1][rt][0][score_gained-1]
                        else:                        
                            state_action[rt][0][score_gained] = noturn_optimal_action_index[0,score_state-score_gained]
                    else:
                        state_action[rt][0][score_gained] = fb.infeasible_marker

            ## policy iteration
            for round_index in range(iteration_round_limit):

                ## policy evaluation
                rt = 3
                score_gained = 0
                score_max_turn = min(score_state-2, 3*fb.maxhitscore)
                prob_turn_transit_nt = solve_turn_transit_probability_fast_nt(score_state, state_action, prob_grid_normalscore_nt, prob_grid_normalscore_t, prob_grid_doublescore_nt, prob_grid_doublescore_t, prob_grid_bullscore_nt, prob_grid_bullscore_t, prob_bust_dic_nt, prob_bust_dic_t)        
                turn_transit_list[score_state].append(prob_turn_transit_nt.copy())
                prob_turn_zeroscore = prob_turn_transit_nt['bust'] + prob_turn_transit_nt['score'][0]
                #pzerolist.append(prob_turn_zeroscore)
                new_value_rt3 = (1 + np.dot(prob_turn_transit_nt['score'][1:], optimal_value_rt3[0,score_state-1:score_state-score_max_turn-1:-1])) / (1-prob_turn_zeroscore)
                state_value_update[rt][0][score_gained] = new_value_rt3
                optimal_value_rt3[0,score_state] = new_value_rt3
                #print('evaluate rt3 value= {}'.format(new_value_rt3)

                ## policy improvement
                for rt in [1,2,3]:            
                    this_throw_state_len = state_len_vector[rt]
                    
                    ## state which can not bust.  score_state-score_gained>=62 
                    state_notbust_len =  max(min(score_state-61, this_throw_state_len),0)
                    if (state_notbust_len > 0):
                        if (rt==1 and round_index==0):
                            ## combine all non-bust states together 
                            state_notbust_update_index = state_notbust_len                    
                            next_state_value_array = np.zeros((61, state_notbust_len))                    
                            for score_gained in range(state_notbust_len):
                                ## skip infeasible state
                                if not fb.state_feasible_array[rt, score_gained]:
                                    continue
                                score_remain = score_state - score_gained
                                score_max = 60 ## always 60 here
                                score_max_plus1 = score_max + 1
                                next_state_value_array[:,score_gained] = optimal_value_rt3[0,score_remain:score_remain-score_max_plus1:-1]
                        elif (rt==2 and (round_index==0 or score_state<182)):
                            ## combine all non-bust states together 
                            state_notbust_update_index = state_notbust_len
                            next_state_value_array = np.zeros((61, state_notbust_len))                    
                            for score_gained in range(state_notbust_len):
                                ## skip infeasible state
                                if not fb.state_feasible_array[rt, score_gained]:
                                    continue
                                score_remain = score_state - score_gained
                                score_max = 60 ## always 60 here
                                score_max_plus1 = score_max + 1
                                next_state_value_array[:,score_gained] = state_value_update[rt-1][0][score_gained:score_gained+score_max_plus1]
                        else: ##(rt==1 and round_index>0) or (rt==2 and round_index>0 and score_state>=182) or (rt==3)
                            ## only update state of score_gained = 0
                            state_notbust_update_index = 1
                            next_state_value_array = np.zeros(61)
                            score_gained = 0
                            score_remain = score_state - score_gained
                            score_max = 60 ## always 60 here
                            score_max_plus1 = score_max + 1                    
                            ## make a copy
                            if (rt > 1):
                                next_state_value_array[:] = state_value_update[rt-1][0][score_gained:score_gained+score_max_plus1]
                            ## transit to next turn when rt=1
                            else:
                                next_state_value_array[:] = optimal_value_rt3[0,score_remain:score_remain-score_max_plus1:-1]

                        ## matrix product to compute all together
                        next_state_value_tensor = torch.from_numpy(next_state_value_array)
                        ## transit to next throw in the same turn when rt=3,2
                        ## -----------TOKEN LOGIC -------------
                        if (rt > 1):                    
                            num_turns_tensor = prob_normalscore_tensor_nt.matmul(next_state_value_tensor)
                        ## transit to next turn when rt=1
                        else:
                            num_turns_tensor = 1 + prob_normalscore_tensor_nt.matmul(next_state_value_tensor)

                        ## searching
                        temp1 = num_turns_tensor.min(axis=0)                
                        state_action_update[rt][0][0:state_notbust_update_index] = temp1.indices.numpy()
                        state_value_update[rt][0][0:state_notbust_update_index] =  temp1.values.numpy()                
                    
                    ## state which possibly bust.  score_state-score_gained<62 
                    if (state_notbust_len < this_throw_state_len):
                        ## combine all bust states together 
                        state_bust_len = this_throw_state_len - state_notbust_len
                        next_state_value_array = np.zeros((61, state_bust_len))
                        for score_gained in range(state_notbust_len, this_throw_state_len):
                            ## skip infeasible state
                            if not fb.state_feasible_array[rt, score_gained]:
                                continue
                            score_remain = score_state - score_gained
                            #score_max = min(score_remain-2, 60)
                            score_max = score_remain-2 ## less than 60 here
                            score_max_plus1 = score_max + 1
                            score_gained_index = score_gained - state_notbust_len ## index off set
                            if (rt > 1):
                                next_state_value_array[0:score_max_plus1,score_gained_index] = state_value_update[rt-1][0][score_gained:score_gained+score_max_plus1]
                            ## transit to next turn when rt=1
                            else:
                                next_state_value_array[0:score_max_plus1,score_gained_index] = optimal_value_rt3[0,score_remain:score_remain-score_max_plus1:-1]
                        
                        next_state_value_tensor = torch.from_numpy(next_state_value_array)
                        ## transit to next throw in the same turn when rt=3,2
                        if (rt > 1):                    
                            num_turns_tensor = prob_normalscore_tensor_nt.matmul(next_state_value_tensor)
                        ## transit to next turn when rt=1
                        else:
                            num_turns_tensor = 1 + prob_normalscore_tensor_nt.matmul(next_state_value_tensor)                                                               

                        ## consider bust/finishing for each bust state seperately 
                        num_turns_array = num_turns_tensor.numpy()                
                        for score_gained in range(state_notbust_len, this_throw_state_len):
                            ## skip infeasible state
                            if not fb.state_feasible_array[rt, score_gained]:
                                continue
                            score_remain = score_state - score_gained
                            #score_max = min(score_remain-2, 60)
                            score_max = score_remain-2 ## less than 60 here
                            score_max_plus1 = score_max + 1
                            score_gained_index = score_gained - state_notbust_len

                            ## transit to the end of game
                            if (rt > 1):
                                if (score_remain == fb.score_DB):                        
                                    num_turns_array[:,score_gained_index] += prob_DB_nt
                                elif (score_remain <= 40 and score_remain%2==0):
                                    num_turns_array[:,score_gained_index] += prob_doublescore_dic_nt[score_remain]
                                else:
                                    pass

                            ## transit to bust
                            if (rt==3):
                                num_turns_array[:,score_gained_index] += prob_bust_dic_nt[score_max]
                                ## solve an equation other than using the policy evaluation value (s,i=3,u=0)
                                num_turns_array[:,score_gained_index] = num_turns_array[:,score_gained_index] / prob_notbust_dic_nt[score_max] 
                            elif (rt==2):
                                num_turns_array[:,score_gained_index] += prob_bust_dic_nt[score_max]*(1+new_value_rt3)
                            else:
                                num_turns_array[:,score_gained_index] += prob_bust_dic_nt[score_max]*(new_value_rt3)  ## 1 turn is already counted before

                        ## searching
                        temp1 = num_turns_tensor.min(axis=0)
                        state_action_update[rt][0][state_notbust_len:this_throw_state_len] = temp1.indices.numpy()
                        state_value_update[rt][0][state_notbust_len:this_throw_state_len] =  temp1.values.numpy()                

                    #### finish rt=1,2,3. check improvement
                    action_diff[rt][0][:] = np.abs(state_action_update[rt][0] - state_action[rt][0])                                
                    value_relerror[rt][0] = np.abs((state_value_update[rt][0] - state_value[rt][0])/state_value_update[rt][0]).max()
                    state_action[rt][0][:] = state_action_update[rt][0][:]
                    state_value[rt][0][:] = state_value_update[rt][0][:]

                max_action_diff = max([action_diff[1].max(), action_diff[2].max(), action_diff[3].max()])
                max_value_relerror = value_relerror.max()            
                
                if (max_action_diff < 1):
                #if max_value_relerror < iteration_relerror_limit:
                    num_iteration_record[score_state] = round_index + 1
                    break

            for rt in [1,2,3]:
                state_value_update[rt][0][fb.state_infeasible[rt]] = fb.largenumber
                state_action_update[rt][0][fb.state_infeasible[rt]] = fb.infeasible_marker
            optimal_action_index_dic[score_state] = ft.copy_numberarray_container(state_action_update)
            optimal_value_dic[score_state] = ft.copy_numberarray_container(state_value_update, new_dtype=fb.result_float_dytpe)
            optimal_value_rt3[0,score_state] = state_value[3][0]
            ## done:V(s,i=3/2/1,u)

##
prob_scorestate_transit = {}    
prob_scorestate_transit =  solve_policy_transit_probability_nt(optimal_action_index_dic, prob_grid_normalscore, prob_grid_doublescore, prob_grid_bullscore)
t2 = time.time()
print('solve dp_turn_policyiter in {} seconds'.format(t2-t1))

print(optimal_value_rt3)
result_dic = {'optimal_value_dic':optimal_value_dic, 'optimal_action_index_dic':optimal_action_index_dic, 'optimal_value_rt3':optimal_value_rt3, 'prob_scorestate_transit':prob_scorestate_transit}


/var/folders/jj/jcq76kp53_zchkx8b5cvhdsh0000gn/T/ipykernel_48695/135309105.py:288: RuntimeWarning: divide by zero encountered in divide
  num_turns_array[:,score_gained_index] = num_turns_array[:,score_gained_index] / prob_notbust_dic_nt[score_max]
/var/folders/jj/jcq76kp53_zchkx8b5cvhdsh0000gn/T/ipykernel_48695/135309105.py:288: RuntimeWarning: overflow encountered in divide
  num_turns_array[:,score_gained_index] = num_turns_array[:,score_gained_index] / prob_notbust_dic_nt[score_max]
/var/folders/jj/jcq76kp53_zchkx8b5cvhdsh0000gn/T/ipykernel_48695/135309105.py:301: RuntimeWarning: divide by zero encountered in divide
  value_relerror[rt][0] = np.abs((state_value_update[rt][0] - state_value[rt][0])/state_value_update[rt][0]).max()
/var/folders/jj/jcq76kp53_zchkx8b5cvhdsh0000gn/T/ipykernel_48695/135309105.py:301: RuntimeWarning: invalid value encountered in divide
  value_relerror[rt][0] = np.abs((state_value_update[rt][0] - state_value[rt][0])/state_value_update[rt][0]).max()


solve prob_policy_transit in 0.2500011920928955 seconds
solve dp_turn_policyiter in 9.978865146636963 seconds
[[0.     0.     1.4381 ... 5.0818 5.0845 5.0965]
 [0.     0.     0.     ... 0.     0.     0.    ]
 [0.     0.     0.     ... 0.     0.     0.    ]
 ...
 [0.     0.     0.     ... 0.     0.     0.    ]
 [0.     0.     0.     ... 0.     0.     0.    ]
 [0.     0.     0.     ... 0.     0.     0.    ]]


In [8]:
prob_turn_transit_nt['bust'] 
#prob_turn_transit_nt['score'][0]

2.0000035272854104

In [106]:
prob_turn_zeroscore


2.1853716285565175

In [112]:
result_dic['optimal_value_rt3'][0][:26]



array([0.0000e+00, 0.0000e+00, 3.5273e-06, 3.4301e-01, 3.7475e-02, 3.4301e-01, 4.9629e-01, 4.9257e-01, 4.9257e-01, 3.2814e-01, 3.4347e-01, 4.0096e-01, 4.7810e-01, 5.7286e-01, 5.8664e-01, 4.9992e-01, 5.5814e-01, 6.0061e-01, 4.7832e-01, 5.1233e-01, 6.6667e-01, 6.6667e-01,        nan,        nan,
              nan,        nan])

In [12]:
prob_turn_transit_nt['bust'] + prob_turn_transit_nt['score'][0]

NameError: name 'prob_turn_transit_nt' is not defined

# Sanity Check for Noturn Solution

In [8]:
[noturn_optimal_value, noturn_optimal_action_index] = fsdt.solve_dp_noturn_tokens(aiming_grid, prob_grid_normalscore_nt, prob_grid_normalscore_t, tokens = tokens, prob_grid_doublescore=prob_grid_doublescore_nt, prob_grid_bullscore=prob_grid_bullscore_nt,prob_grid_doublescore_t=prob_grid_doublescore_t, prob_grid_bullscore_t=prob_grid_bullscore_t)


/Users/rachael/Desktop/OptimalDarts-main/function_solve_dp_tokens.py:174: RuntimeWarning: overflow encountered in divide
  num_tothrow = num_tothrow / prob_otherstate


In [148]:
prob_bust_dic


{0: array([1., 1., 1., ..., 0., 0., 0.]),
 1: array([0.9611, 0.7815, 0.7229, ..., 0.    , 0.    , 0.    ]),
 2: array([0.9611, 0.7815, 0.7229, ..., 0.    , 0.    , 0.    ]),
 3: array([0.9577, 0.7815, 0.7229, ..., 0.    , 0.    , 0.    ]),
 4: array([0.9577, 0.7815, 0.7228, ..., 0.    , 0.    , 0.    ]),
 5: array([0.8981, 0.5099, 0.4768, ..., 0.    , 0.    , 0.    ]),
 6: array([0.8981, 0.5099, 0.4768, ..., 0.    , 0.    , 0.    ]),
 7: array([0.8981, 0.5099, 0.4768, ..., 0.    , 0.    , 0.    ]),
 8: array([0.8981, 0.5099, 0.4768, ..., 0.    , 0.    , 0.    ]),
 9: array([0.8981, 0.5099, 0.476 , ..., 0.    , 0.    , 0.    ]),
 10: array([0.8981, 0.5099, 0.476 , ..., 0.    , 0.    , 0.    ]),
 11: array([0.8981, 0.5099, 0.476 , ..., 0.    , 0.    , 0.    ]),
 12: array([0.898 , 0.5075, 0.443 , ..., 0.    , 0.    , 0.    ]),
 13: array([0.898 , 0.5075, 0.443 , ..., 0.    , 0.    , 0.    ]),
 14: array([0.898 , 0.5075, 0.4423, ..., 0.    , 0.    , 0.    ]),
 15: array([0.898 , 0.5075, 0

In [6]:
def solve_turn_transit_probability_fast(score_state, state_action, prob_normalscore, prob_doublescore, prob_bullscore, prob_bust_dic):
    """
    A fast way of implementing solve_turn_transit_probability by using pre-stored prob_bust_dic
    """     
    
    result_dict = {}
    prob_finish = 0 ## probability of finishing the game
    prob_bust_total = 0   ## probability of busting the game
    ## initialize for (s, rt=3, score_gained=0)
    next_throw_state_len = 1
    prob_transit_next_throw_state = np.ones(next_throw_state_len)
    
    for rt in [3,2,1]:
        prob_this_throw_state = prob_transit_next_throw_state
        this_throw_state_len = next_throw_state_len
        next_throw_state_len = min(score_state-2, fb.maxhitscore*(4-rt)) + 1
        prob_transit_next_throw_state = np.zeros(next_throw_state_len)  ## probability vector of total score_gained after this throw
        
        prob_normalscore_transit = prob_normalscore[state_action[rt][0:this_throw_state_len]]*prob_this_throw_state.reshape((this_throw_state_len,1))
        
        for score_gained in range(this_throw_state_len):  # loop through score already gained
            ## skip infeasible state
            if not fb.state_feasible_array[rt, score_gained]:
                continue   

            ## aimming location of the policy at this state
            aiming_location_index = state_action[rt][score_gained]
            prob_this_state = prob_this_throw_state[score_gained]
            
            #largest possible normal socre to make in the next throw without busting
            score_remain = score_state - score_gained
            score_max = min(score_remain-2, 60)
            score_max_plus1 = score_max + 1
        
            ## transit to next throw or turn with normal scores            
            prob_transit_next_throw_state[score_gained:score_gained+score_max_plus1] += prob_normalscore_transit[score_gained, 0:score_max_plus1]
            ## game can not bust or end when score_max = 60, i.e.,  prob_notbust = 1
            if (score_max < 60):
                ## transit to the end of game
                if (score_remain == fb.score_DB):
                    prob_finish += prob_bullscore[aiming_location_index, 1]*prob_this_state
                elif (score_remain <= 40 and score_remain%2==0):
                    doublescore_index = (score_remain//2) - 1
                    prob_finish += prob_doublescore[aiming_location_index, doublescore_index]*prob_this_state
                else:
                    pass

                #transit to bust
                prob_bust_total += prob_bust_dic[score_max][aiming_location_index]*prob_this_state
            
    result_dict['finish'] = prob_finish
    result_dict['bust'] = prob_bust_total
    result_dict['score'] = prob_transit_next_throw_state

    return result_dict 

In [7]:
#pzerolist = []
## aiming_grid
num_aiming_location = aiming_grid.shape[0]
prob_normalscore = prob_grid_normalscore
prob_doublescore_dic = {}
for doublescore_index in range(20):
    doublescore = 2*(doublescore_index+1)
    prob_doublescore_dic[doublescore] = np.array(prob_grid_doublescore[:,doublescore_index])
prob_DB = np.array(prob_grid_bullscore[:,1])

## the probability of not bust for each action given score_max=i (score_remain=i+2)
prob_bust_dic = {}
prob_notbust_dic = {}
for score_max in range(60):    
    ## transit to next throw or turn
    prob_notbust = prob_grid_normalscore[:,0:score_max+1].sum(axis=1)
    ## transit to the end of game
    score_remain = score_max + 2
    if (score_remain == fb.score_DB):
        prob_notbust += prob_DB
    elif (score_remain <= 40 and score_remain%2==0):
        prob_notbust += prob_doublescore_dic[score_remain]
    ##
    prob_notbust = np.minimum(np.maximum(prob_notbust, 0),1)
    prob_notbust_dic[score_max] = prob_notbust
    prob_bust_dic[score_max] = 1 - prob_notbust_dic[score_max]

prob_normalscore_tensor = torch.from_numpy(prob_normalscore)

iteration_round_limit = 20
iteration_relerror_limit = 10**-9

#### state space example of (SB=25 DB=50) ####
## rt: the number of remaining throws in a turn
## state_infeasible_rt2 = [23, 29, 31, 35, 37, 41, 43, 44, 46, 47, 49, 52, 53, 55, 56, 58, 59]
## state_infeasible_rt1 = [103, 106, 109, 112, 113, 115, 116, 118, 119]    
    
optimal_value_rt3 = np.zeros(502) #vector: optimal value for the beginning state of each turn (rt=3)
optimal_value_dic = {} ## first key: score=0,2,...,501, second key: remaining throws=3,2,1
optimal_action_index_dic = {}
num_iteration_record = np.zeros(502, dtype=np.int32)

state_len_vector = np.zeros(4, dtype=np.int32)
state_value  = [None]  ## optimal value (expected # of turns to finish the game) for each state in the current playing turn
state_action = [None]  ## aimming locations for for each state in the current playing turn
action_diff  = [None]
value_relerror = np.zeros(4)
for rt in [1,2,3]:
    ## for rt=3: possible score_gained = 0
    ## for rt=2: possible score_gained = 0,1,...,60
    ## for rt=1: possible score_gained = 0,1,...,120
    this_throw_state_len = fb.maxhitscore*(3-rt) + 1
    state_value.append(np.ones(this_throw_state_len)*fb.largenumber)
    state_action.append(np.ones(this_throw_state_len, np.int32)*fb.infeasible_marker)
    action_diff.append(np.ones(this_throw_state_len))
state_value_update = ft.copy_numberarray_container(state_value)
state_action_update = ft.copy_numberarray_container(state_action)

## use no_turn policy as the initial policy
[noturn_optimal_value, noturn_optimal_action_index] = fsdt.solve_dp_noturn(aiming_grid, prob_grid_normalscore, prob_grid_doublescore, prob_grid_bullscore)

t1 = time.time()
for score_state in range(2, 502):
    #print('#### solve_dp_turn score_state={} ####'.format(score_state))    
    
    ## initialization 
    for rt in [1,2,3]:
        ## for rt=3: score_gained = 0
        ## for rt=2: score_gained = 0,1,...,min(s-2,60)
        ## for rt=1: score_gained = 0,1,...,min(s-2,120)
        this_throw_state_len = min(score_state-2, fb.maxhitscore*(3-rt)) + 1
        state_len_vector[rt] = this_throw_state_len
                
        ## initialize the starting policy: 
        ## use no_turn action in (s, i, u=0)
        ## use turn action (s-1, i, u-1) in (s, i, u!=0) if (s-1, i, u-1) is feasible state
        state_action[rt][0] = noturn_optimal_action_index[score_state]            
        for score_gained in range(1,this_throw_state_len):                
            if fb.state_feasible_array[rt, score_gained]:  ## if True
                if fb.state_feasible_array[rt, score_gained-1]:
                    state_action[rt][score_gained] = optimal_action_index_dic[score_state-1][rt][score_gained-1]
                else:                        
                    state_action[rt][score_gained] = noturn_optimal_action_index[score_state-score_gained]
            else:
                state_action[rt][score_gained] = fb.infeasible_marker

    ## policy iteration
    for round_index in range(iteration_round_limit):

        ## policy evaluation
        rt = 3
        score_gained = 0
        score_max_turn = min(score_state-2, 3*fb.maxhitscore)
        prob_turn_transit = solve_turn_transit_probability_fast(score_state, state_action, prob_grid_normalscore, prob_grid_doublescore, prob_grid_bullscore, prob_bust_dic)        
        prob_turn_zeroscore = prob_turn_transit['bust'] + prob_turn_transit['score'][0]
        #pzerolist.append(prob_turn_zeroscore)
        new_value_rt3 = (1 + np.dot(prob_turn_transit['score'][1:], optimal_value_rt3[score_state-1:score_state-score_max_turn-1:-1])) / (1-prob_turn_zeroscore)
        state_value_update[rt][score_gained] = new_value_rt3
        optimal_value_rt3[score_state] = new_value_rt3
        #print('evaluate rt3 value= {}'.format(new_value_rt3)

        ## policy improvement
        for rt in [1,2,3]:            
            this_throw_state_len = state_len_vector[rt]
            
            ## state which can not bust.  score_state-score_gained>=62 
            state_notbust_len =  max(min(score_state-61, this_throw_state_len),0)
            if (state_notbust_len > 0):
                if (rt==1 and round_index==0):
                    ## combine all non-bust states together 
                    state_notbust_update_index = state_notbust_len                    
                    next_state_value_array = np.zeros((61, state_notbust_len))                    
                    for score_gained in range(state_notbust_len):
                        ## skip infeasible state
                        if not fb.state_feasible_array[rt, score_gained]:
                            continue
                        score_remain = score_state - score_gained
                        score_max = 60 ## always 60 here
                        score_max_plus1 = score_max + 1
                        next_state_value_array[:,score_gained] = optimal_value_rt3[score_remain:score_remain-score_max_plus1:-1]
                elif (rt==2 and (round_index==0 or score_state<182)):
                    ## combine all non-bust states together 
                    state_notbust_update_index = state_notbust_len
                    next_state_value_array = np.zeros((61, state_notbust_len))                    
                    for score_gained in range(state_notbust_len):
                        ## skip infeasible state
                        if not fb.state_feasible_array[rt, score_gained]:
                            continue
                        score_remain = score_state - score_gained
                        score_max = 60 ## always 60 here
                        score_max_plus1 = score_max + 1
                        next_state_value_array[:,score_gained] = state_value_update[rt-1][score_gained:score_gained+score_max_plus1]
                else: ##(rt==1 and round_index>0) or (rt==2 and round_index>0 and score_state>=182) or (rt==3)
                    ## only update state of score_gained = 0
                    state_notbust_update_index = 1
                    next_state_value_array = np.zeros(61)
                    score_gained = 0
                    score_remain = score_state - score_gained
                    score_max = 60 ## always 60 here
                    score_max_plus1 = score_max + 1                    
                    ## make a copy
                    if (rt > 1):
                        next_state_value_array[:] = state_value_update[rt-1][score_gained:score_gained+score_max_plus1]
                    ## transit to next turn when rt=1
                    else:
                        next_state_value_array[:] = optimal_value_rt3[score_remain:score_remain-score_max_plus1:-1]

                ## matrix product to compute all together
                next_state_value_tensor = torch.from_numpy(next_state_value_array)
                ## transit to next throw in the same turn when rt=3,2
                if (rt > 1):                    
                    num_turns_tensor = prob_normalscore_tensor.matmul(next_state_value_tensor)
                ## transit to next turn when rt=1
                else:
                    num_turns_tensor = 1 + prob_normalscore_tensor.matmul(next_state_value_tensor)

                ## searching
                temp1 = num_turns_tensor.min(axis=0)                
                state_action_update[rt][0:state_notbust_update_index] = temp1.indices.numpy()
                state_value_update[rt][0:state_notbust_update_index] =  temp1.values.numpy()                
            
            ## state which possibly bust.  score_state-score_gained<62 
            if (state_notbust_len < this_throw_state_len):
                ## combine all bust states together 
                state_bust_len = this_throw_state_len - state_notbust_len
                next_state_value_array = np.zeros((61, state_bust_len))
                for score_gained in range(state_notbust_len, this_throw_state_len):
                    ## skip infeasible state
                    if not fb.state_feasible_array[rt, score_gained]:
                        continue
                    score_remain = score_state - score_gained
                    #score_max = min(score_remain-2, 60)
                    score_max = score_remain-2 ## less than 60 here
                    score_max_plus1 = score_max + 1
                    score_gained_index = score_gained - state_notbust_len ## index off set
                    if (rt > 1):
                        next_state_value_array[0:score_max_plus1,score_gained_index] = state_value_update[rt-1][score_gained:score_gained+score_max_plus1]
                    ## transit to next turn when rt=1
                    else:
                        next_state_value_array[0:score_max_plus1,score_gained_index] = optimal_value_rt3[score_remain:score_remain-score_max_plus1:-1]
                
                next_state_value_tensor = torch.from_numpy(next_state_value_array)
                ## transit to next throw in the same turn when rt=3,2
                if (rt > 1):                    
                    num_turns_tensor = prob_normalscore_tensor.matmul(next_state_value_tensor)
                ## transit to next turn when rt=1
                else:
                    num_turns_tensor = 1 + prob_normalscore_tensor.matmul(next_state_value_tensor)                                                               

                ## consider bust/finishing for each bust state seperately 
                num_turns_array = num_turns_tensor.numpy()                
                for score_gained in range(state_notbust_len, this_throw_state_len):
                    ## skip infeasible state
                    if not fb.state_feasible_array[rt, score_gained]:
                        continue
                    score_remain = score_state - score_gained
                    #score_max = min(score_remain-2, 60)
                    score_max = score_remain-2 ## less than 60 here
                    score_max_plus1 = score_max + 1
                    score_gained_index = score_gained - state_notbust_len

                    ## transit to the end of game
                    if (rt > 1):
                        if (score_remain == fb.score_DB):                        
                            num_turns_array[:,score_gained_index] += prob_DB
                        elif (score_remain <= 40 and score_remain%2==0):
                            num_turns_array[:,score_gained_index] += prob_doublescore_dic[score_remain]
                        else:
                            pass

                    ## transit to bust
                    if (rt==3):
                        num_turns_array[:,score_gained_index] += prob_bust_dic[score_max]
                        ## solve an equation other than using the policy evaluation value (s,i=3,u=0)
                        num_turns_array[:,score_gained_index] = num_turns_array[:,score_gained_index] / prob_notbust_dic[score_max] 
                    elif (rt==2):
                        num_turns_array[:,score_gained_index] += prob_bust_dic[score_max]*(1+new_value_rt3)
                    else:
                        num_turns_array[:,score_gained_index] += prob_bust_dic[score_max]*(new_value_rt3)  ## 1 turn is already counted before

                ## searching
                temp1 = num_turns_tensor.min(axis=0)
                state_action_update[rt][state_notbust_len:this_throw_state_len] = temp1.indices.numpy()
                state_value_update[rt][state_notbust_len:this_throw_state_len] =  temp1.values.numpy()                

            #### finish rt=1,2,3. check improvement
            action_diff[rt][:] = np.abs(state_action_update[rt] - state_action[rt])                                
            value_relerror[rt] = np.abs((state_value_update[rt] - state_value[rt])/state_value_update[rt]).max()
            state_action[rt][:] = state_action_update[rt][:]
            state_value[rt][:] = state_value_update[rt][:]

        max_action_diff = max([action_diff[1].max(), action_diff[2].max(), action_diff[3].max()])
        max_value_relerror = value_relerror.max()            
        
        if (max_action_diff < 1):
        #if max_value_relerror < iteration_relerror_limit:
            num_iteration_record[score_state] = round_index + 1
            break

    for rt in [1,2,3]:
        state_value_update[rt][fb.state_infeasible[rt]] = fb.largenumber
        state_action_update[rt][fb.state_infeasible[rt]] = fb.infeasible_marker
    optimal_action_index_dic[score_state] = ft.copy_numberarray_container(state_action_update)
    optimal_value_dic[score_state] = ft.copy_numberarray_container(state_value_update, new_dtype=fb.result_float_dytpe)
    optimal_value_rt3[score_state] = state_value[3][0]
    ## done:V(s,i=3/2/1,u)

##
prob_scorestate_transit = {}    
prob_scorestate_transit =  fep.solve_policy_transit_probability(optimal_action_index_dic, prob_grid_normalscore, prob_grid_doublescore, prob_grid_bullscore)
t2 = time.time()
print('solve dp_turn_policyiter in {} seconds'.format(t2-t1))

print(optimal_value_rt3)
result_dic = {'optimal_value_dic':optimal_value_dic, 'optimal_action_index_dic':optimal_action_index_dic, 'optimal_value_rt3':optimal_value_rt3, 'prob_scorestate_transit':prob_scorestate_transit}



/Users/rachael/Desktop/OptimalDarts-main/function_solve_dp_tokens.py:82: RuntimeWarning: divide by zero encountered in divide
  num_tothrow = num_tothrow / prob_otherstate
/Users/rachael/Desktop/OptimalDarts-main/function_solve_dp_tokens.py:82: RuntimeWarning: overflow encountered in divide
  num_tothrow = num_tothrow / prob_otherstate
/var/folders/jj/jcq76kp53_zchkx8b5cvhdsh0000gn/T/ipykernel_48695/636321877.py:215: RuntimeWarning: divide by zero encountered in divide
  num_turns_array[:,score_gained_index] = num_turns_array[:,score_gained_index] / prob_notbust_dic[score_max]
/var/folders/jj/jcq76kp53_zchkx8b5cvhdsh0000gn/T/ipykernel_48695/636321877.py:215: RuntimeWarning: overflow encountered in divide
  num_turns_array[:,score_gained_index] = num_turns_array[:,score_gained_index] / prob_notbust_dic[score_max]
/var/folders/jj/jcq76kp53_zchkx8b5cvhdsh0000gn/T/ipykernel_48695/636321877.py:228: RuntimeWarning: divide by zero encountered in divide
  value_relerror[rt] = np.abs((state_val

solve prob_policy_transit in 0.24787569046020508 seconds
solve dp_turn_policyiter in 1.7931010723114014 seconds
[0.     0.     1.4381 ... 5.0818 5.0845 5.0965]


In [142]:
prob_notbust

array([0.9934, 1.    , 1.    , ..., 1.    , 1.    , 1.    ])